## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings;warnings.filterwarnings(action='ignore')

## Read data

In [2]:
train = pd.read_csv('../data/stayed_train.csv')
test = pd.read_csv('../data/stayed_test.csv')

In [3]:
print('학습데이터 수:', train.shape)
print('평가데이터 수:', test.shape)

학습데이터 수: (127741, 19)
평가데이터 수: (40148, 18)


In [4]:
'''
데이터 설명
- sessionID : 세션 ID
- userID : 사용자 ID
- TARGET : 세션에서 발생한 총 조회수
- browser : 사용된 브라우저
- OS : 사용된 기기의 운영체제
- device : 사용된 기기
- new : 첫 방문 여부 (0: 첫 방문 아님, 1: 첫 방문)
- quality : 세션의 질 (거래 성사를 기준으로 측정된 값, 범위: 1~100)
- duration : 총 세션 시간 (단위: 초)
- bounced : 이탈 여부 (0: 이탈하지 않음, 1: 이탈함)
- transaction : 세션 내에서 발생의 거래의 수
- transaction_revenue : 총 거래 수익
- continent : 세션이 발생한 대륙
- subcontinent : 세션이 발생한 하위 대륙
- country : 세션이 발생한 국가
- traffic_source : 트래픽이 발생한 소스
- traffic_medium : 트래픽 소스의 매체
- keyword : 트래픽 소스의 키워드, 일반적으로 traffic_medium이 organic, cpc인 경우에 설정
- referral_path : traffic_medium이 referral인 경우 설정되는 경로
'''
display(train.head())

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000007,USER_000007,5.0,Chrome,Macintosh,desktop,1,1.0,64.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
3,SESSION_000008,USER_000008,5.0,Firefox,Linux,desktop,1,1.0,60.0,0,0.0,0.0,Americas,South America,Brazil,youtube.com,referral,NaN,Category5_Path_0002
4,SESSION_000009,USER_000009,3.0,Chrome,Macintosh,desktop,1,2.0,579.0,0,0.0,0.0,Americas,Northern America,United States,google,cpc,Category1_Keyword_000,NaN


### 결측치
- keyword, referral_path에는 traffic_medium값에 따른 결측치가 존재한다.


In [5]:
train.isna().sum()

sessionID                  0
userID                     0
TARGET                     0
browser                    0
OS                         0
device                     0
new                        0
quality                    0
duration                   0
bounced                    0
transaction                0
transaction_revenue        0
continent                  0
subcontinent               0
country                    0
traffic_source             0
traffic_medium             0
keyword                70399
referral_path          79591
dtype: int64

In [6]:
# keyword, referral_path의 값을 결정하는 traffic_medium의 빈도를 살펴보니 결측치로 판단할 수 있는 (none), (not set)이 있다.
train.traffic_medium.value_counts()

organic      55247
(none)       35686
referral     30738
cpc           3416
affiliate     2477
cpm            174
(not set)        3
Name: traffic_medium, dtype: int64

In [7]:
# (none)은 직접 트래픽이라 트래픽 매체가 없는 것이고 (not set)은 결측치로 봐야한다.
print('(none)의 트래픽 소스:', train.query('traffic_medium == "(none)"').traffic_source.unique())
print('(not set)의 트래픽 소스:', train.query('traffic_medium == "(not set)"').traffic_source.unique())

(none)의 트래픽 소스: ['(direct)']
(not set)의 트래픽 소스: ['Partners' 'google']


In [8]:
# (not set)으로 결측치가 표현된 다른 변수가 있는지 확인한 결과 OS, continent, subcontinent, country에서도 나타난다.
train.applymap(lambda x: x=="(not set)").sum()

sessionID                 0
userID                    0
TARGET                    0
browser                   0
OS                     1044
device                    0
new                       0
quality                   0
duration                  0
bounced                   0
transaction               0
transaction_revenue       0
continent               148
subcontinent            148
country                 148
traffic_source            0
traffic_medium            3
keyword                   0
referral_path             0
dtype: int64

In [9]:
# traffic_medium 중 광고로 접속한 cpc, cpm은 ad(광고)로 변경한다.
train['traffic_medium'] = train['traffic_medium'].replace('cpc','ad').replace('cpm','ad')
test['traffic_medium'] = test['traffic_medium'].replace('cpc','ad').replace('cpm','ad')

# (not set)은 결측치로 학습데이터 내 트래픽 소스의 최빈값으로 채운다.
mode = train.groupby('traffic_source')['traffic_medium'].agg(lambda x: x.mode()[0])
ease = train.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
train.loc[ease.index, 'traffic_medium'] = ease.values

ease = test.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
test.loc[ease.index, 'traffic_medium'] = ease.values

### 변수 정리

In [10]:
train[['keyword1','keyword2']] = train['keyword'].str.split('_', expand=True)[[0,2]]
test[['keyword1','keyword2']] = test['keyword'].str.split('_', expand=True)[[0,2]]

train[['keyword1','keyword2']].fillna("(not set)", inplace=True)
test[['keyword1','keyword2']].fillna("(not set)", inplace=True)

In [11]:
train[['referral_path1','referral_path2']] = train['referral_path'].str.split('_', expand=True)[[0,2]]
test[['referral_path1','referral_path2']] = test['referral_path'].str.split('_', expand=True)[[0,2]]

train[['referral_path1','referral_path2']].fillna("(not set)", inplace=True)
test[['referral_path1','referral_path2']].fillna("(not set)", inplace=True)

In [12]:
# 불필요한 열 제거
train.drop(['keyword', 'referral_path'], axis=1, inplace=True)
test.drop(['keyword', 'referral_path'], axis=1, inplace=True)

### 이상치 제거

In [13]:
outlier = [train.transaction.sort_values().index[-1], train.TARGET.sort_values().index[-1]]
train.drop(outlier, inplace=True)
print('학습데이터 크기:', train.shape)

학습데이터 크기: (127739, 21)


## Save data

In [14]:
train.to_csv('../data/stayed_train.csv', index=False)
test.to_csv('../data/stayed_train.csv', index=False)